# Regression analysis for vaccination uptake rates

## Import packages

In [202]:
import pandas as pd
import libpysal
import matplotlib.pyplot as plt
import mgwr
from mgwr.gwr import GWR, MGWR
import numpy as np
import pysal
from pysal import model
import geopandas as gpd
import esda

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import PartialDependenceDisplay, partial_dependence

from matplotlib import colors

import copy

from importlib import reload
import run_OLS
reload(run_OLS)

import multiprocessing as mp

import utilities_plot
reload(utilities_plot)

import pickle

## Set up parallel computing for GWR

In [2]:
#This might be needed to turn off the OpenMP multi-threading
%env OMP_NUM_THREADS = 1

env: OMP_NUM_THREADS=1


In [3]:
#Parrallelization is more favored when you your data are large and/or your machine have many many cores.
#mgwr has soft dependency of numba, please install numba if you need better performance (pip install numba).

n_proc = 8 #two processors
pool = mp.Pool(n_proc) 

## Import shapefile

Download and read MSOA shapefile data.

If the MSOA shapefile is not there, run the next cell to download it.

In [4]:
# note that this dataset contains all 7201 MSOAs in England. This dataset is used as it contains the high resolution boundary and neighbour topology of MSOAs 
# url = 'https://github.com/jreades/fsds/raw/master/data/src/Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip'

#! wget $url

In [5]:
# path_data_folder = '../Data'
gdf = gpd.read_file(f"../Data/Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip!Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2.shp")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7201 entries, 0 to 7200
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    7201 non-null   int64   
 1   MSOA11CD    7201 non-null   object  
 2   MSOA11NM    7201 non-null   object  
 3   MSOA11NMW   7201 non-null   object  
 4   BNG_E       7201 non-null   int64   
 5   BNG_N       7201 non-null   int64   
 6   LONG        7201 non-null   float64 
 7   LAT         7201 non-null   float64 
 8   Shape__Are  7201 non-null   float64 
 9   Shape__Len  7201 non-null   float64 
 10  geometry    7201 non-null   geometry
dtypes: float64(4), geometry(1), int64(3), object(3)
memory usage: 619.0+ KB


In [6]:
gdf.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

## Import uptake data

In [7]:
# import df_uptake data
df_uptake = pd.read_csv("../Data/vaccine_uptake_socioeco.csv")
df_uptake = df_uptake.rename({'E2sfca_30_min_supply_value':'accessibility_vaccine'}, axis = 1)

In [8]:
df_uptake.columns

Index(['Unnamed: 0', 'MSOA', '18over1st_dose', '18over2nd_dose', 'pop0_17',
       'pop18over', 'vaccination_percentage_1stdose',
       'vaccination_percentage_2nddose', 'vaccination_percentage_total',
       'MSOA Code', 'pct_pop_18_29', 'pct_pop_30_39', 'pct_pop_40_49',
       'pct_pop_50_59', 'pct_pop_60_69', 'pct_pop_70_80', 'pct_pop_80_over',
       'MSOA code', 'average_household_income', 'IMD19 SCORE',
       'msoa_imd_decile', 'pct_White', 'pct_Mixed', 'pct_Asian', 'pct_black',
       'pct_other', 'pct_hh_car', 'msoa_quintile', 'MSOA11CD', 'income_score',
       'employ_score', 'edu_score', 'health_score', 'crime_score',
       'housing_score', 'livEnv_score', 'accessibility_vaccine'],
      dtype='object')

In [12]:
df_uptake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6787 entries, 0 to 6786
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      6787 non-null   int64  
 1   MSOA                            6787 non-null   object 
 2   18over1st_dose                  6787 non-null   int64  
 3   18over2nd_dose                  6787 non-null   int64  
 4   pop0_17                         6787 non-null   int64  
 5   pop18over                       6787 non-null   int64  
 6   vaccination_percentage_1stdose  6787 non-null   float64
 7   vaccination_percentage_2nddose  6787 non-null   float64
 8   vaccination_percentage_total    6787 non-null   float64
 9   MSOA Code                       6787 non-null   object 
 10  pct_pop_18_29                   6787 non-null   float64
 11  pct_pop_30_39                   6787 non-null   float64
 12  pct_pop_40_49                   67

In [13]:
# one-hot encoding but keeping the original column
series_msoa_decile = df_uptake.msoa_imd_decile
# one-hot encoding of MSOADECILE
df_uptake = pd.get_dummies(df_uptake, prefix=['msoa_imd_decile'], columns=['msoa_imd_decile'])
df_uptake=df_uptake.assign(msoa_imd_decile = series_msoa_decile)

In [14]:
df_uptake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6787 entries, 0 to 6786
Data columns (total 47 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      6787 non-null   int64  
 1   MSOA                            6787 non-null   object 
 2   18over1st_dose                  6787 non-null   int64  
 3   18over2nd_dose                  6787 non-null   int64  
 4   pop0_17                         6787 non-null   int64  
 5   pop18over                       6787 non-null   int64  
 6   vaccination_percentage_1stdose  6787 non-null   float64
 7   vaccination_percentage_2nddose  6787 non-null   float64
 8   vaccination_percentage_total    6787 non-null   float64
 9   MSOA Code                       6787 non-null   object 
 10  pct_pop_18_29                   6787 non-null   float64
 11  pct_pop_30_39                   6787 non-null   float64
 12  pct_pop_40_49                   67

In [15]:
df_uptake['vaccination_percentage_2nddose'].head()

0    0.999152
1    0.961992
2    0.979597
3    0.910034
4    0.910515
Name: vaccination_percentage_2nddose, dtype: float64

In [16]:
df_uptake.columns

Index(['Unnamed: 0', 'MSOA', '18over1st_dose', '18over2nd_dose', 'pop0_17',
       'pop18over', 'vaccination_percentage_1stdose',
       'vaccination_percentage_2nddose', 'vaccination_percentage_total',
       'MSOA Code', 'pct_pop_18_29', 'pct_pop_30_39', 'pct_pop_40_49',
       'pct_pop_50_59', 'pct_pop_60_69', 'pct_pop_70_80', 'pct_pop_80_over',
       'MSOA code', 'average_household_income', 'IMD19 SCORE', 'pct_White',
       'pct_Mixed', 'pct_Asian', 'pct_black', 'pct_other', 'pct_hh_car',
       'msoa_quintile', 'MSOA11CD', 'income_score', 'employ_score',
       'edu_score', 'health_score', 'crime_score', 'housing_score',
       'livEnv_score', 'accessibility_vaccine', 'msoa_imd_decile_1',
       'msoa_imd_decile_2', 'msoa_imd_decile_3', 'msoa_imd_decile_4',
       'msoa_imd_decile_5', 'msoa_imd_decile_6', 'msoa_imd_decile_7',
       'msoa_imd_decile_8', 'msoa_imd_decile_9', 'msoa_imd_decile_10',
       'msoa_imd_decile'],
      dtype='object')

In [17]:
df_uptake.vaccination_percentage_2nddose.hist()

<AxesSubplot:>

## Creating queen's weight matrix

In [18]:
# gdf: only keep the MSOAs in the df_uptake
gdf_england = gdf.loc[gdf.MSOA11CD.isin(df_uptake.MSOA)]
# Calculating neighbours based on the Queen's contiguity
wq = libpysal.weights.Queen.from_dataframe(gdf_england)

/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)


In [19]:
# Now we can visualise the weights in the map
f, ax = plt.subplots(1,1, figsize=(10,10))
gdf_england.plot(ax=ax)
# wq.plot(gdf_england_no_London, ax=ax)

<AxesSubplot:>

In [20]:
df_uptake.columns

Index(['Unnamed: 0', 'MSOA', '18over1st_dose', '18over2nd_dose', 'pop0_17',
       'pop18over', 'vaccination_percentage_1stdose',
       'vaccination_percentage_2nddose', 'vaccination_percentage_total',
       'MSOA Code', 'pct_pop_18_29', 'pct_pop_30_39', 'pct_pop_40_49',
       'pct_pop_50_59', 'pct_pop_60_69', 'pct_pop_70_80', 'pct_pop_80_over',
       'MSOA code', 'average_household_income', 'IMD19 SCORE', 'pct_White',
       'pct_Mixed', 'pct_Asian', 'pct_black', 'pct_other', 'pct_hh_car',
       'msoa_quintile', 'MSOA11CD', 'income_score', 'employ_score',
       'edu_score', 'health_score', 'crime_score', 'housing_score',
       'livEnv_score', 'accessibility_vaccine', 'msoa_imd_decile_1',
       'msoa_imd_decile_2', 'msoa_imd_decile_3', 'msoa_imd_decile_4',
       'msoa_imd_decile_5', 'msoa_imd_decile_6', 'msoa_imd_decile_7',
       'msoa_imd_decile_8', 'msoa_imd_decile_9', 'msoa_imd_decile_10',
       'msoa_imd_decile'],
      dtype='object')

# Regression analysis: OLS, GWR, MGWR

## Data preparation

In [32]:
# add coords to df_uptake by linking to gdf_england_no_London
if 'BNG_E' not in df_uptake.columns.values:
    df_uptake = df_uptake.merge(gdf_england[['MSOA11CD', 'BNG_E', 'BNG_N']], left_on = 'MSOA', right_on='MSOA11CD', how='left')
df_uptake.columns

Index(['Unnamed: 0', 'MSOA', '18over1st_dose', '18over2nd_dose', 'pop0_17',
       'pop18over', 'vaccination_percentage_1stdose',
       'vaccination_percentage_2nddose', 'vaccination_percentage_total',
       'MSOA Code', 'pct_pop_18_29', 'pct_pop_30_39', 'pct_pop_40_49',
       'pct_pop_50_59', 'pct_pop_60_69', 'pct_pop_70_80', 'pct_pop_80_over',
       'MSOA code', 'average_household_income', 'IMD19 SCORE', 'pct_White',
       'pct_Mixed', 'pct_Asian', 'pct_black', 'pct_other', 'pct_hh_car',
       'msoa_quintile', 'MSOA11CD_x', 'income_score', 'employ_score',
       'edu_score', 'health_score', 'crime_score', 'housing_score',
       'livEnv_score', 'accessibility_vaccine', 'msoa_imd_decile_1',
       'msoa_imd_decile_2', 'msoa_imd_decile_3', 'msoa_imd_decile_4',
       'msoa_imd_decile_5', 'msoa_imd_decile_6', 'msoa_imd_decile_7',
       'msoa_imd_decile_8', 'msoa_imd_decile_9', 'msoa_imd_decile_10',
       'msoa_imd_decile', 'MSOA11CD_y', 'BNG_E', 'BNG_N'],
      dtype='object')

In the variables of age group proportions, the group of 30-39 and 40-49 have the largest proportions.
In the variabels of ethnicity, the group of white has the largest value.

In [33]:
list_x_var = ['msoa_quintile',
              'pct_pop_18_29', 
#               'pct_pop_30_39', 
              'pct_pop_40_49',
              'pct_pop_50_59', 
              'pct_pop_60_69', 
              'pct_pop_70_80', 
              'pct_pop_80_over',
              'pct_Mixed' , 
              'pct_Asian' , 
              'pct_black' , 
#               'pct_White',
              'pct_other',
              'average_household_income',
              'pct_hh_car',
              'accessibility_vaccine'
             ]

In [34]:
#Prepare dataset inputs
g_y = df_uptake['vaccination_percentage_2nddose'].values.reshape((-1,1))
g_X = df_uptake[list_x_var].values

u = df_uptake['BNG_E'].astype(float)
v = df_uptake['BNG_N'].astype(float)
g_coords = list(zip(u,v))

# Standardised our data to have mean of 0 and standard deviation of 1
g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)

g_y = g_y.reshape((-1,1))

g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

g_X = np.array(g_X).astype(float)
g_y = np.array(g_y).astype(float)

## OLS and VIF

In [35]:
# OLS regression
ols = run_OLS.run_OLS(g_X, g_y, wq, list_x_var)

,AIC,Adj_R2,RSS,Log-likelihood
0,11117.140902,0.699476,2035.449906,-5543.570451


,Variable,Coefficient,Std.Error,t-Statistic,p-value,VIF
0,CONSTANT,-3.046836e-17,0.006655,-4.578427e-15,1.000000e+00,NaN
1,msoa_quintile,-3.570125e-02,0.014505,-2.461371e+00,1.386546e-02,4.750585
2,pct_pop_18_29,-1.669582e-01,0.007699,-2.168652e+01,6.983796e-101,1.338351
3,pct_pop_40_49,1.930723e-01,0.017052,1.132241e+01,1.867738e-29,6.565936
4,pct_pop_50_59,1.293320e-01,0.008868,1.458457e+01,1.836440e-47,1.775657
5,pct_pop_60_69,1.624228e-01,0.015611,1.040446e+01,3.658822e-25,5.502875
6,pct_pop_70_80,2.708510e-01,0.020774,1.303786e+01,2.154151e-38,9.745009
7,pct_pop_80_over,1.400802e-01,0.012662,1.106305e+01,3.302648e-28,3.620243
8,pct_Mixed,-1.261302e-01,0.014388,-8.766128e+00,2.308744e-18,4.674732
9,pct_Asian,3.729447e-02,0.009294,4.012789e+00,6.065226e-05,1.950431


## GWR

### Notes

Please note that selecting the bandwidths for GWR and MGWR takes a long time. You can use a small subset of the data in pilot studies (see the use of max_ind).

### Select the bandwidth

In [36]:
# the variable of max_ind is used to select some or all rows for model calibration
# if you want to use a small subset, you can set max_ind = 100
# if you want to use the whole dataset, set max_ind = None

max_ind = None
# set it as None if all rows are used
# reference: https://mgwr.readthedocs.io/en/latest/generated/mgwr.sel_bw.Sel_BW.html

gwr_selector = mgwr.sel_bw.Sel_BW(g_coords[:max_ind], 
                  g_y[:max_ind], # Dependent variable
                  g_X[:max_ind], # Independent variable
                  multi=False)

In [37]:
%%time
# mgwr_bw = mgwr_selector.search(multi_bw_max=[200])
gwr_bw = gwr_selector.search(pool=pool) #add pool to Sel_BW.search
print(gwr_bw)

195.0
CPU times: user 2.72 s, sys: 422 ms, total: 3.14 s
Wall time: 8min 9s


In [38]:
# mgwr_selector_no_limit_max_bandwidth is the copy of mgwr_selector, for selecting bandwidth without max bandwidth limit
# mgwr_selector_no_limit_max_bandwidth = copy.deepcopy(mgwr_selector)

In [39]:
df_bandwidth = pd.DataFrame({'var' : ['Intercept'] + list_x_var, 'bandwidth' : gwr_bw.tolist()})
display(df_bandwidth)

,var,bandwidth
0,Intercept,195.0
1,msoa_quintile,195.0
2,pct_pop_18_29,195.0
3,pct_pop_40_49,195.0
4,pct_pop_50_59,195.0
5,pct_pop_60_69,195.0
6,pct_pop_70_80,195.0
7,pct_pop_80_over,195.0
8,pct_Mixed,195.0
9,pct_Asian,195.0


### Build the model

In [150]:
%%time
# build the GWR model
gwr_result = GWR(g_coords, g_y, g_X, gwr_bw).fit(pool=pool) #add pool to GWR.fit
results = gwr_result

CPU times: user 214 ms, sys: 51.3 ms, total: 265 ms
Wall time: 45.9 s


In [177]:
results = gwr_result
print(results.summary())

Model type                                                         Gaussian
Number of observations:                                                6787
Number of covariates:                                                    15

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           2035.450
Log-likelihood:                                                   -5543.570
AIC:                                                              11117.141
AICc:                                                             11119.221
BIC:                                                             -57712.310
R2:                                                                   0.700
Adj. R2:                                                              0.699

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

### Testing the spatial autocorrelation of residuals

In [178]:
results.resid_response

array([ 0.47889928, -0.35420096,  0.57973388, ..., -0.29076592,
        0.05544995, -0.11977835])

In [179]:
mi = esda.moran.Moran(results.resid_response, w=wq, two_tailed=False)
# pysal.model.spreg.diagnostics_sp.MoranRes(, w=wq)

In [180]:
print("P value under normality assumption:")
print("%.5f"%mi.p_norm)
print("P value via permutation:")
print("%.5f"%mi.p_sim)
print("P value under randomization assumption:")
print("%.5f"%mi.p_rand)

P value under normality assumption:
0.00005
P value via permutation:
0.00100
P value under randomization assumption:
0.00005


From our GWR model, we can get the local estimates as well as local R2.

It is important in GWR that we only include statistically significant estimates (in our case, we will assign 95% confidence intervals, thus the p-value of our local estimates need to be lower than 0.05.

Luckily, mgwr package has a method to extract only a filtered set of significant local estimates by assigning 0 for local estimates that are not significant using filter_tval.

In [181]:
# Filtering only significant result at 0.05 (95% confidence interval)
filtered_estimates = results.filter_tvals(alpha=.05)
filtered_estimates

# Can you check the estimates using 99% and 90% confidence interval
filtered_estimates90 = results.filter_tvals(alpha=.1)
filtered_estimates99 = results.filter_tvals(alpha=.01)

### Visualising the results

In [182]:
# Convert arrays to data frame
data_params = pd.DataFrame(filtered_estimates)
# data_localR2 = pd.DataFrame(results.localR2)
data_resid = pd.DataFrame(results.resid_response)

In [183]:
list_x_var

['msoa_quintile',
 'pct_pop_18_29',
 'pct_pop_40_49',
 'pct_pop_50_59',
 'pct_pop_60_69',
 'pct_pop_70_80',
 'pct_pop_80_over',
 'pct_Mixed',
 'pct_Asian',
 'pct_black',
 'pct_other',
 'average_household_income',
 'pct_hh_car',
 'accessibility_vaccine']

In [184]:
# Create the new dataframe
df1=pd.DataFrame(df_uptake["MSOA"])
df2 = df1.assign(intercept=data_params[0],
                 msoa_quintile=data_params[1],
                 pct_pop_18_29=data_params[2],
#                  pct_pop_30_39=data_params[3],
                 pct_pop_40_49=data_params[3],
                 pct_pop_50_59=data_params[4],
                 pct_pop_60_69=data_params[5],
                 pct_pop_70_80=data_params[6],
                 pct_pop_80_over=data_params[7],
                 pct_Mixed=data_params[8],
                 pct_Asian=data_params[9],
                 pct_black=data_params[10],
                 pct_other=data_params[11],
                 average_household_income=data_params[12],
                 pct_hh_car=data_params[13],
                 accessibility_vaccine=data_params[14],
#                  eth_other=data_params[15],
                 resid = data_resid[0],
#                  localR2=data_localR2[0],
                )
df2.columns
df2

,MSOA,intercept,msoa_quintile,pct_pop_18_29,pct_pop_40_49,pct_pop_50_59,pct_pop_60_69,pct_pop_70_80,pct_pop_80_over,pct_Mixed,pct_Asian,pct_black,pct_other,average_household_income,pct_hh_car,accessibility_vaccine,resid
0,E02002796,0.00000,0.0,0.000000,0.000000,2.028707,0.0,0.000000,4.700679,0.000000,0.000000,-2.005177,0.000000,0.0,3.199313,0.000000,0.478899
1,E02002797,0.00000,0.0,0.000000,0.000000,2.169001,0.0,0.000000,4.583291,0.000000,0.000000,0.000000,0.000000,0.0,3.198982,0.000000,-0.354201
2,E02002798,0.00000,0.0,0.000000,2.440066,2.445809,0.0,2.370863,5.341833,0.000000,0.000000,0.000000,0.000000,0.0,2.838295,-2.715342,0.579734
3,E02002799,0.00000,0.0,0.000000,2.114518,2.465011,0.0,2.081100,5.217638,0.000000,0.000000,0.000000,0.000000,0.0,2.993155,-2.378772,0.793734
4,E02002800,0.00000,0.0,0.000000,0.000000,2.277814,0.0,0.000000,4.786062,0.000000,0.000000,0.000000,0.000000,0.0,3.083792,0.000000,-0.079892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6782,E02002478,-2.07743,0.0,2.183395,2.476020,2.048728,0.0,2.173161,3.249685,0.000000,0.000000,0.000000,-3.496839,0.0,0.000000,0.000000,-0.092628
6783,E02002479,0.00000,0.0,2.235101,2.130381,2.308333,0.0,0.000000,3.074763,0.000000,-2.269625,0.000000,-2.805899,0.0,0.000000,0.000000,-0.084567
6784,E02002480,0.00000,0.0,2.588593,2.142960,2.793647,0.0,0.000000,3.803785,-2.124959,0.000000,0.000000,-2.084401,0.0,0.000000,0.000000,-0.290766
6785,E02002481,0.00000,0.0,2.688043,2.148930,2.764284,0.0,0.000000,3.869216,-2.326644,-2.070547,0.000000,0.000000,0.0,0.000000,0.000000,0.055450


Merge the GWR results with the shapefile.

In [185]:
# Merge our shapefile with the model's results using left_join
gdf_gwr_result = gdf_england.merge(df2, left_on='MSOA11CD', right_on='MSOA')
gdf_gwr_result.head()

,OBJECTID,MSOA11CD,MSOA11NM,MSOA11NMW,BNG_E,BNG_N,LONG,LAT,Shape__Are,Shape__Len,...,pct_pop_70_80,pct_pop_80_over,pct_Mixed,pct_Asian,pct_black,pct_other,average_household_income,pct_hh_car,accessibility_vaccine,resid
0,2,E02000002,Barking and Dagenham 001,Barking and Dagenham 001,548267,189685,0.138756,51.58652,2.166163e+06,8150.405928,...,3.674717,0.0,0.0,2.846316,-3.271971,0.0,3.067400,-3.389619,0.0,-0.002511
1,3,E02000003,Barking and Dagenham 002,Barking and Dagenham 002,548259,188520,0.138149,51.57605,2.143568e+06,9118.196243,...,3.528651,0.0,0.0,2.864193,-3.343343,0.0,2.936867,-3.154693,0.0,-0.240941
2,4,E02000004,Barking and Dagenham 003,Barking and Dagenham 003,551004,186412,0.176828,51.55638,2.491467e+06,8206.551627,...,3.520212,0.0,0.0,2.351956,-3.320960,0.0,3.248946,-2.985057,0.0,-0.059519
3,5,E02000005,Barking and Dagenham 004,Barking and Dagenham 004,548733,186824,0.144267,51.56069,1.186053e+06,6949.688798,...,3.419248,0.0,0.0,2.780169,-3.442852,0.0,3.146859,-2.993731,0.0,0.409840
4,6,E02000007,Barking and Dagenham 006,Barking and Dagenham 006,549698,186609,0.158087,51.55850,1.733891e+06,6773.520925,...,3.504710,0.0,0.0,2.632310,-3.341324,0.0,3.256147,-3.031404,0.0,-0.303851


In [186]:
# visualise the residuals
# f,ax=plt.subplots(2,3,figsize=(15,6), subplot_kw=dict(aspect='equal'))
# # Flatten them
# ax = ax.flatten()

# (gdf_gwr_result
#  .sort_values('resid')
#  .plot('resid',
#        ax=ax,
#        legend=True,
#        vmin=np.min(gdf_gwr_result.resid),
#        vmax=np.max(gdf_gwr_result.resid),
#        cmap='Reds'))

# ax.set_xticklabels([])
# ax.set_yticklabels([])
# ax.set_xticks([])
# ax.set_yticks([])

# ax.set_title('residual', fontsize=16)
    
# f.tight_layout()
    
# plt.show()

f, ax = plt.subplots(1,1, figsize=(10,10))
ax.set_title('MGWR standardised residual', fontsize=16)
# gwr_resid = results.resid_response
gdf_gwr_result.plot('resid',
       ax=ax,
       legend=True,
       vmin=np.min(gdf_gwr_result.resid),
       vmax=np.max(gdf_gwr_result.resid),
       cmap='Reds')

<AxesSubplot:title={'center':'MGWR standardised residual'}>

In [187]:
# For the last one, local R2, we can just map all of them regarding the significance

# (gdf_gwr_result
#  .sort_values('localR2')
#  .plot('localR2',
#        ax=ax[-1],
#        legend=True,
#        vmin=0,
#        vmax=1,
#        cmap='Reds'))
    
# ax[-1].set_xticklabels([])
# ax[-1].set_yticklabels([])
# ax[-1].set_xticks([])
# ax[-1].set_yticks([])
    
# ax[-1].set_title('Local R2', fontsize=16)

In [188]:
# GWR coefficients of IMD decile, hh_car, average_income, accessibility
utilities_plot.plot_car_income_access_imd_coef(gdf_gwr_result)
# plt.show()
plt.savefig('../Images/GWR_coef_car_income_accessibility_imd_quintile.png', bbox_inches='tight')

In [189]:
# Ethnic composition
utilities_plot.plot_ethnic_coef(gdf_gwr_result, b_same_value_range = True)
# plt.show()
plt.savefig('../Images/GWR_coef_ethnic_mixed_asian_black_other_1.png', bbox_inches='tight')
utilities_plot.plot_ethnic_coef(gdf_gwr_result, b_same_value_range = False)
# plt.show()
plt.savefig('../Images/GWR_coef_ethnic_mixed_asian_black_other_2.png', bbox_inches='tight')

-7.340055140946626
6.775745655474381
None
None


In [190]:
# Age group
utilities_plot.plot_age_group_coef(gdf_gwr_result, b_same_value_range = True)
# plt.show()
plt.savefig('../Images/GWR_coef_age_group_1.png', bbox_inches='tight')
utilities_plot.plot_age_group_coef(gdf_gwr_result, b_same_value_range = False)
# plot_age_group_coef_custom_legend(b_same_value_range = False)
# plt.show()
plt.savefig('../Images/GWR_coef_age_group_2.png', bbox_inches='tight')

-10.088976185944446
9.661935515736317
None
None


## MGWR

### Select the bandwidth

In [55]:
# Calibrate MGWR model

# the variable of max_ind is used to select some or all rows for model calibration
# if you want to use a small subset, you can set max_ind = 100
# if you want to use the whole dataset, set max_ind = None
max_ind = None

# reference: https://mgwr.readthedocs.io/en/latest/generated/mgwr.sel_bw.Sel_BW.html

mgwr_bandwidth_selector = mgwr.sel_bw.Sel_BW(g_coords[:max_ind], 
                  g_y[:max_ind], # Dependent variable
                  g_X[:max_ind], # Independent variable
                  multi=True)

In [58]:
%%time
# mgwr_bw = mgwr_selector.search(multi_bw_max=[200])
mgwr_bandwidth = mgwr_bandwidth_selector.search(pool=pool)
print(mgwr_bandwidth)

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

[  43.  233.  110. 1319.  158. 5052.  456. 1315. 4334.  289. 6643.  738.
 6786. 6786. 6786.]
CPU times: user 33min 6s, sys: 1min 49s, total: 34min 56s
Wall time: 1d 4min 33s


In [128]:
df_bandwidth = pd.DataFrame({'var' : ['Intercept'] + list_x_var, 'bandwidth' : mgwr_bandwidth.tolist()})
display(df_bandwidth)

,var,bandwidth
0,Intercept,43.0
1,msoa_quintile,233.0
2,pct_pop_18_29,110.0
3,pct_pop_40_49,1319.0
4,pct_pop_50_59,158.0
5,pct_pop_60_69,5052.0
6,pct_pop_70_80,456.0
7,pct_pop_80_over,1315.0
8,pct_Mixed,4334.0
9,pct_Asian,289.0


### Build the model

In [62]:
%%time
# build the GWR model
mgwr_result = MGWR(g_coords, g_y, g_X, mgwr_bandwidth_selector).fit(pool=pool) #add pool to GWR.fit
results = mgwr_result

Inference:   0%|          | 0/8 [00:00<?, ?it/s]

CPU times: user 17.8 s, sys: 4.29 s, total: 22.1 s
Wall time: 4h 39min 7s


In [203]:
results = mgwr_result
results.summary()

Model type                                                         Gaussian
Number of observations:                                                6787
Number of covariates:                                                    15

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           2035.450
Log-likelihood:                                                   -5543.570
AIC:                                                              11117.141
AICc:                                                             11119.221
BIC:                                                             -57712.310
R2:                                                                   0.700
Adj. R2:                                                              0.699

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

### Testing the spatial autocorrelation of residuals

From our GWR model, we can get the local estimates as well as local R2.

It is important in GWR that we only include statistically significant estimates (in our case, we will assign 95% confidence intervals, thus the p-value of our local estimates need to be lower than 0.05.

Luckily, mgwr package has a method to extract only a filtered set of significant local estimates by assigning 0 for local estimates that are not significant using filter_tval.

In [204]:
# Filtering only significant result at 0.05 (95% confidence interval)
filtered_estimates = results.filter_tvals(alpha=.05)
filtered_estimates

# Can you check the estimates using 99% and 90% confidence interval
filtered_estimates90 = results.filter_tvals(alpha=.1)
filtered_estimates99 = results.filter_tvals(alpha=.01)

### Visualising the results

In [205]:
# Convert arrays to data frame
data_params = pd.DataFrame(filtered_estimates)
# data_localR2 = pd.DataFrame(results.localR2)
data_resid = pd.DataFrame(results.resid_response)

In [206]:
list_x_var

['msoa_quintile',
 'pct_pop_18_29',
 'pct_pop_40_49',
 'pct_pop_50_59',
 'pct_pop_60_69',
 'pct_pop_70_80',
 'pct_pop_80_over',
 'pct_Mixed',
 'pct_Asian',
 'pct_black',
 'pct_other',
 'average_household_income',
 'pct_hh_car',
 'accessibility_vaccine']

In [207]:
# Create the new dataframe
df1=pd.DataFrame(df_uptake["MSOA"])
df2 = df1.assign(intercept=data_params[0],
                 msoa_quintile=data_params[1],
                 pct_pop_18_29=data_params[2],
#                  pct_pop_30_39=data_params[3],
                 pct_pop_40_49=data_params[3],
                 pct_pop_50_59=data_params[4],
                 pct_pop_60_69=data_params[5],
                 pct_pop_70_80=data_params[6],
                 pct_pop_80_over=data_params[7],
                 pct_Mixed=data_params[8],
                 pct_Asian=data_params[9],
                 pct_black=data_params[10],
                 pct_other=data_params[11],
                 average_household_income=data_params[12],
                 pct_hh_car=data_params[13],
                 accessibility_vaccine=data_params[14],
#                  eth_other=data_params[15],
                 resid = data_resid[0],
#                  localR2=data_localR2[0],
                )
df2.columns
df2

,MSOA,intercept,msoa_quintile,pct_pop_18_29,pct_pop_40_49,pct_pop_50_59,pct_pop_60_69,pct_pop_70_80,pct_pop_80_over,pct_Mixed,pct_Asian,pct_black,pct_other,average_household_income,pct_hh_car,accessibility_vaccine,resid
0,E02002796,3.194496,-3.754599,0.000000,13.732469,4.112776,11.040620,11.740689,10.567993,-3.682870,0.000000,-2.903293,-7.295733,6.960070,17.688857,2.870373,0.034053
1,E02002797,3.783179,-3.663887,0.000000,13.725388,4.187470,11.032273,11.817030,10.639315,-3.681422,0.000000,-2.901962,-7.345193,6.959432,17.687561,2.869522,-0.240282
2,E02002798,3.083787,-3.858957,0.000000,13.838504,4.252472,11.026666,11.904370,10.649775,-3.696240,0.000000,-2.913417,-7.473528,6.962209,17.690970,2.870083,0.359583
3,E02002799,3.518577,-3.791276,0.000000,13.812377,4.224297,11.026537,11.899242,10.655423,-3.691918,0.000000,-2.910502,-7.454690,6.961448,17.689954,2.869811,0.645044
4,E02002800,3.727246,-3.733656,0.000000,13.762274,4.188346,11.029792,11.857600,10.645018,-3.686271,0.000000,-2.905971,-7.401236,6.960380,17.688684,2.869654,-0.332242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6782,E02002478,0.000000,0.000000,2.031059,10.016552,3.034068,11.324262,8.538504,9.615891,-3.660700,-4.099501,-2.852598,-2.332852,6.959993,17.706676,2.894328,-0.176532
6783,E02002479,0.000000,0.000000,2.015908,9.922748,3.187368,11.325779,8.150518,9.335235,-3.653704,-3.496864,-2.840957,-2.235094,6.957347,17.703617,2.894151,-0.249460
6784,E02002480,0.000000,0.000000,0.000000,10.061812,3.472427,11.318343,8.545715,9.485099,-3.656281,-3.546813,-2.849513,-2.497617,6.958911,17.704877,2.893572,-0.242858
6785,E02002481,0.000000,0.000000,0.000000,10.118821,3.532251,11.316347,8.650677,9.566712,-3.658155,-3.517952,-2.853101,-2.560173,6.959646,17.705600,2.893464,0.119880


Merge the MGWR results with the shapefile.

In [208]:
# Merge our shapefile with the model's results using left_join
gdf_gwr_result = gdf_england.merge(df2, left_on='MSOA11CD', right_on='MSOA')
gdf_gwr_result.head()

,OBJECTID,MSOA11CD,MSOA11NM,MSOA11NMW,BNG_E,BNG_N,LONG,LAT,Shape__Are,Shape__Len,...,pct_pop_70_80,pct_pop_80_over,pct_Mixed,pct_Asian,pct_black,pct_other,average_household_income,pct_hh_car,accessibility_vaccine,resid
0,2,E02000002,Barking and Dagenham 001,Barking and Dagenham 001,548267,189685,0.138756,51.58652,2.166163e+06,8150.405928,...,0.0,5.123424,-6.859484,2.152820,-3.123284,-8.219458,7.042267,17.749934,2.853867,0.013504
1,3,E02000003,Barking and Dagenham 002,Barking and Dagenham 002,548259,188520,0.138149,51.57605,2.143568e+06,9118.196243,...,0.0,5.121742,-6.861392,2.164850,-3.123292,-8.247824,7.042569,17.750248,2.854298,-0.382897
2,4,E02000004,Barking and Dagenham 003,Barking and Dagenham 003,551004,186412,0.176828,51.55638,2.491467e+06,8206.551627,...,0.0,5.193086,-6.865779,2.161754,-3.124292,-8.434863,7.043608,17.751292,2.854565,-0.426421
3,5,E02000005,Barking and Dagenham 004,Barking and Dagenham 004,548733,186824,0.144267,51.56069,1.186053e+06,6949.688798,...,0.0,5.134029,-6.863843,2.180370,-3.123469,-8.302177,7.043094,17.750786,2.854832,0.337765
4,6,E02000007,Barking and Dagenham 006,Barking and Dagenham 006,549698,186609,0.158087,51.55850,1.733891e+06,6773.520925,...,0.0,5.158213,-6.864929,2.170386,-3.123841,-8.357768,7.043323,17.751012,2.854733,-0.101952


In [209]:
# visualise the residuals

f, ax = plt.subplots(1,1, figsize=(10,10))
ax.set_title('MGWR standardised residual', fontsize=16)
# gwr_resid = results.resid_response
gdf_gwr_result.plot('resid',
       ax=ax,
       legend=True,
       vmin=np.min(gdf_gwr_result.resid),
       vmax=np.max(gdf_gwr_result.resid),
       cmap='Reds')

<AxesSubplot:title={'center':'MGWR standardised residual'}>

In [210]:
# For the last one, local R2, we can just map all of them regarding the significance

# (gdf_gwr_result
#  .sort_values('localR2')
#  .plot('localR2',
#        ax=ax[-1],
#        legend=True,
#        vmin=0,
#        vmax=1,
#        cmap='Reds'))
    
# ax[-1].set_xticklabels([])
# ax[-1].set_yticklabels([])
# ax[-1].set_xticks([])
# ax[-1].set_yticks([])
    
# ax[-1].set_title('Local R2', fontsize=16)

In [211]:
# GWR coefficients of IMD decile, hh_car, average_income, accessibility
utilities_plot.plot_car_income_access_imd_coef(gdf_gwr_result)
# plt.show()
plt.savefig('../Images/MGWR_coef_car_income_accessibility_imd_quintile.png', bbox_inches='tight')

/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


In [212]:
# Ethnic composition
utilities_plot.plot_ethnic_coef(gdf_gwr_result, b_same_value_range = True)
# plt.show()
plt.savefig('../Images/MGWR_coef_ethnic_mixed_asian_black_other_1.png', bbox_inches='tight')
utilities_plot.plot_ethnic_coef(gdf_gwr_result, b_same_value_range = False)
# plt.show()
plt.savefig('../Images/MGWR_coef_ethnic_mixed_asian_black_other_2.png', bbox_inches='tight')

-9.667458355029991
8.97266542364267


/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


None
None


/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


In [213]:
# Age group
utilities_plot.plot_age_group_coef(gdf_gwr_result, b_same_value_range = True)
# plt.show()
plt.savefig('../Images/MGWR_coef_age_group_1.png', bbox_inches='tight')
utilities_plot.plot_age_group_coef(gdf_gwr_result, b_same_value_range = False)
# plot_age_group_coef_custom_legend(b_same_value_range = False)
# plt.show()
plt.savefig('../Images/MGWR_coef_age_group_2.png', bbox_inches='tight')

-11.256503251713948
14.605392799668756


/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


None
None


/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/geopandas/plotting.py:684: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


## Summary of three models

In [123]:
# demo code of GWR
mi = esda.moran.Moran(ols.u, w=wq, two_tailed=False)

# pysal.model.spreg.diagnostics_sp.MoranRes(, w=wq)
print("P value under normality assumption:")
print("%.5f"%mi.p_norm)
print("P value via permutation:")
print("%.5f"%mi.p_sim)
print("P value under randomization assumption:")
print("%.5f"%mi.p_rand)

P value under normality assumption:
0.00000
P value via permutation:
0.00100
P value under randomization assumption:
0.00000


In [144]:
summary_OLS = run_OLS.model_summary(ols, wq)
summary_gwr = run_OLS.model_summary(gwr_result, wq)
summary_mgwr = run_OLS.model_summary(mgwr_result, wq)
summary_models = pd.concat([summary_OLS,summary_gwr, summary_mgwr], axis=0)
summary_models.index = ['OLS', 'GWR', 'MGWR']
display(summary_models)

,AIC,Adj_R2,RSS,Log-likelihood,morans_I,z_score,p_val
OLS,11117.140902,0.699476,2035.449906,-5543.570451,0.202791,27.108641,0.000000
GWR,7981.687675,0.837708,910.622808,-2814.031164,0.029087,3.905153,0.000047
MGWR,7914.267420,0.831629,1009.074178,-3162.407075,0.010331,1.399894,0.080773


In [132]:
mgwr_result.pvalues

array([[ 3.19449615, -3.75459893, -0.11669305, ...,  6.96007049,
        17.6888567 ,  2.87037294],
       [ 3.78317927, -3.66388748, -0.49085721, ...,  6.95943205,
        17.68756134,  2.86952191],
       [ 3.08378742, -3.85895685,  0.78126693, ...,  6.96220879,
        17.69097   ,  2.87008343],
       ...,
       [-0.79184159, -0.55661581,  1.84671968, ...,  6.95891136,
        17.70487713,  2.89357208],
       [-0.74825853, -0.42518756,  1.72205291, ...,  6.95964619,
        17.70559954,  2.89346406],
       [-0.97167662, -0.64574675,  1.86137727, ...,  6.95817486,
        17.70398251,  2.89346874]])

In [120]:
# pickle the gwr and mgwr results
file_pickle_GWR = 'GWR_result.pickle'
file_pickle_MGWR = 'MGWR_result.pickle'

with open(file_pickle_GWR, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(gwr_result, f, pickle.HIGHEST_PROTOCOL)
with open(file_pickle_MGWR, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(mgwr_result, f, pickle.HIGHEST_PROTOCOL)

You can load the pickle files of the GWR and MGWR object.

In [ ]:
# # load these pickles
# file_pickle_GWR = 'GWR_result.pickle'
# file_pickle_MGWR = 'MGWR_result.pickle'
# with open(file_pickle_GWR, 'rb') as f:
#     gwr_result = pickle.load(f)
# with open(file_pickle_MGWR, 'rb') as f:
#     mgwr_result = pickle.load(f)